In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn # to remove warnings

import pandas as pd, numpy as np, matplotlib.pyplot as plt
% matplotlib notebook

In [2]:
original_data = pd.read_csv ('train.csv')
final_test_set = pd.read_csv ('test.csv') 
'''
The target values aren't given (they're on the Kaggle Website), 
so the final_test_set is not used to check for overfitting.
'''
original_data.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [3]:
original_data.shape

(9557, 143)

In [4]:
description = original_data.describe()  
description.round (2)

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
count,2697.00,9557.00,9557.00,9557.00,9557.00,9557.00,9557.00,2215.00,9557.00,9557.00,...,9557.00,9557.00,9557.00,9557.00,9557.00,9557.00,9557.00,9552.00,9557.00,9557.00
mean,165231.61,0.04,4.96,0.02,0.99,0.96,0.23,1.40,0.39,1.56,...,74.22,1643.77,19.13,53.50,3.84,3.25,3.90,102.59,1643.77,3.30
std,150457.13,0.19,1.47,0.15,0.07,0.20,0.42,0.76,0.68,1.04,...,76.78,1741.20,18.75,78.45,6.95,4.13,12.51,93.52,1741.20,1.01
min,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.04,0.00,0.00,0.00,1.00
25%,80000.00,0.00,4.00,0.00,1.00,1.00,0.00,1.00,0.00,1.00,...,16.00,289.00,9.00,0.00,0.00,1.00,0.11,36.00,289.00,3.00
50%,130000.00,0.00,5.00,0.00,1.00,1.00,0.00,1.00,0.00,1.00,...,36.00,961.00,16.00,36.00,1.00,2.25,0.44,81.00,961.00,4.00
75%,200000.00,0.00,6.00,0.00,1.00,1.00,0.00,2.00,1.00,2.00,...,121.00,2601.00,25.00,81.00,4.00,4.00,1.78,134.56,2601.00,4.00
max,2353477.00,1.00,11.00,1.00,1.00,1.00,1.00,6.00,5.00,8.00,...,441.00,9409.00,169.00,441.00,81.00,36.00,64.00,1369.00,9409.00,4.00


In [6]:
def create_dict_of_var_and_description (file_name):
    ''' This is used to create a dictionary from the Kaggle document that describes all the variables '''
    open_file = open(file_name)
    file = open_file.read()
    open_file.close()
    dict = {}
    for var_descrip in file.split('\n'):
        var, description = var_descrip.split(',', 1)
        dict[var] = description
    return dict

var_file_name = 'Var_name_and_description'

dict_of_var = create_dict_of_var_and_description (var_file_name)
dict_of_var['v2a1'] # example

' Monthly rent payment'

In [18]:
def find_columns_with_string_values (df):
    ''' 
    This prints the variable that contains strings (it can contain strings and numbers), 
    the description, and the unique values.    
    '''
    dtypes_of_data = pd.DataFrame (df.dtypes)
    list_of_str_var = dtypes_of_data [dtypes_of_data == object].dropna().index
    for var in list_of_str_var[1:]: # ignore the first column 'Id' because that was not in the Kaggle Variable File
         print (var, ':', dict_of_var[var], '\n', df[var].unique())
find_columns_with_string_values (original_data)

idhogar :  Household level identifier 
 ['21eb7fcc1' '0e5d7a658' '2c7317ea8' ... 'a8eeafc29' '212db6f6c'
 'd6c086aa3']
dependency :  Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64) 
 ['no' '8' 'yes' '3' '.5' '.25' '2' '.66666669' '.33333334' '1.5'
 '.40000001' '.75' '1.25' '.2' '2.5' '1.2' '4' '1.3333334' '2.25'
 '.22222222' '5' '.83333331' '.80000001' '6' '3.5' '1.6666666' '.2857143'
 '1.75' '.71428573' '.16666667' '.60000002']
edjefe :  years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0 
 ['10' '12' 'no' '11' '9' '15' '4' '6' '8' '17' '7' '16' '14' '5' '21' '2'
 '19' 'yes' '3' '18' '13' '20']
edjefa :  years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0 
 ['no' '11' '4' '10' '9' '15' '7' 

In [19]:
from sklearn.model_selection import train_test_split
train, test = train_test_split (original_data, test_size=0.2, random_state=42)
X, y = train.drop('Target', axis=1), train['Target'].copy()
X_test, y_test = test.drop('Target', axis=1), test['Target'].copy()

In [23]:
def deal_with_string_values (X):
    ''' 
    For example, the columns that contain no and yes are dependency, edjefe, edjefa.
    For edjefe and edjefa, yes means 1, and no means 0.
    For dependency, We assume that when people answered the survey, no means 0 and yes means the average.
    '''
    replaced_strings_df = X.replace ('no', 0).replace ('yes', np.nan).drop (['Id', 'idhogar'], axis=1)
    replaced_strings_df = replaced_strings_df.fillna ({'edjefe': 1, 'edjefa': 1})
    from sklearn.preprocessing import Imputer
    imputer = Imputer(strategy='median')
    replace_NA_with_median = imputer.fit_transform (replaced_strings_df)
    return pd.DataFrame (replace_NA_with_median, columns = replaced_strings_df.columns)

def drop_variables_with_low_corr_with_target (X, original_data, threshold):
    ''' If the correlation of a certain variable with the target is less than the threshold,
    we drop the variables from the dataframe'''
    train = original_data.copy()
    corr_matrix = train.corr()
    corr_target = corr_matrix['Target'].sort_values(ascending=False)
    over_threshold_var = corr_target[corr_target.abs() < threshold].index
    return X.drop (over_threshold_var, axis=1), over_threshold_var

def Clean_Data (X, original_data, threshold):
    ''' Combines the above two functions, and returns a dataframe and the dropped variables
    (so we can drop the same variables during the test set, and the final test set)'''
    replace_string_transform = deal_with_string_values (X)
    drop_low_corr, dropped_var = drop_variables_with_low_corr_with_target (replace_string_transform, original_data, threshold)
    return drop_low_corr, dropped_var

''' We use standard scaler to account for different scales, and imputer to save the median.
As a result, we could transform the test data using the median, variance, and mean from the train
set, and not that of the test set.
'''
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler
trans_pipeline = Pipeline([
        ('imputer', Imputer(strategy='median')),
        ('std_scaler', StandardScaler())
])
    
X_Cleaned, Dropped_Var = Clean_Data (X, original_data, .1)
X_Cleaned_and_transformed = pd.DataFrame (trans_pipeline.fit_transform (X_Cleaned), columns = X_Cleaned.columns)
X_Cleaned_and_transformed.head()

,v2a1,hacdor,rooms,hacapo,refrig,v18q,r4h1,r4h2,r4m1,r4m3,...,mobilephone,qmobilephone,lugar1,age,SQBescolari,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBmeaned
0,-0.126508,-0.201397,-0.654642,-0.15878,0.210372,-0.544245,-0.571774,-1.505326,-0.572567,0.773699,...,0.15704,0.113060,-1.191667,-0.802546,0.342669,-0.540920,-0.680766,0.020975,-0.241685,-0.706485
1,1.474548,-0.201397,-0.654642,-0.15878,0.210372,-0.544245,-0.571774,-0.543020,0.876862,-0.053101,...,0.15704,-0.562210,0.839161,-0.201338,0.618502,-0.540920,-0.218579,-0.410087,-0.241685,-0.317323
2,-0.126508,-0.201397,0.026090,-0.15878,0.210372,1.837409,-0.571774,0.419286,-0.572567,-0.053101,...,0.15704,0.788331,0.839161,-0.663806,-0.497965,-0.172741,-0.680766,-0.410087,0.174260,0.285953
3,0.006913,-0.201397,-0.654642,-0.15878,0.210372,-0.544245,0.885782,-0.543020,-0.572567,-0.053101,...,0.15704,0.113060,0.839161,0.076142,-0.497965,-0.172741,-0.051678,0.020975,0.174260,-0.639388
4,0.313782,-0.201397,0.706821,-0.15878,0.210372,-0.544245,-0.571774,1.381592,-0.572567,-0.879901,...,0.15704,0.788331,-1.191667,-0.432572,0.618502,-0.172741,-0.218579,-0.410087,-0.353924,1.216365


In [24]:
def display_var_with_missing_values (description):
    var = description.copy()
    count = description.loc['count']
    with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
        return pd.DataFrame ((count [count != 9557.0]))
    
def get_description_of_values_of_var_with_missing_values(description, file_name):
    df_of_var_with_missing_values = display_var_with_missing_values (description)
    list_of_var = list (display_var_with_missing_values (description).index) 
    dict_of_var = create_dict_of_var_and_description (file_name)
    for var in list_of_var:
        print (var, ':', dict_of_var[var])
        
df_of_var_with_missing_values = display_var_with_missing_values (description)  
df_of_var_with_missing_values

,count
v2a1,2697.0
v18q1,2215.0
rez_esc,1629.0
meaneduc,9552.0
SQBmeaned,9552.0


In [25]:
X_clean_descr = X_Cleaned.describe()  
X_clean_descr.round(4)

,v2a1,hacdor,rooms,hacapo,refrig,v18q,r4h1,r4h2,r4m1,r4m3,...,mobilephone,qmobilephone,lugar1,age,SQBescolari,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBmeaned
count,7645.0000,7645.0000,7645.0000,7645.0000,7645.0000,7645.0000,7645.0000,7645.0000,7645.000,7645.0000,...,7645.0000,7645.0000,7645.0000,7645.0000,7645.0000,7645.0000,7645.0000,7645.000,7645.0000,7645.0000
mean,139481.8709,0.0390,4.9617,0.0246,0.9576,0.2285,0.3923,1.5643,0.395,2.0642,...,0.9759,2.8326,0.5868,34.3536,73.9116,19.2842,53.0252,3.854,3.2668,101.8082
std,74955.4109,0.1936,1.4691,0.1549,0.2015,0.4199,0.6861,1.0392,0.690,1.2096,...,0.1533,1.4810,0.4924,21.6245,76.1380,19.0137,77.8956,6.960,4.2076,93.1549
min,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.000,0.0400,0.0000
25%,130000.0000,0.0000,4.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.000,1.0000,...,1.0000,2.0000,0.0000,17.0000,16.0000,9.0000,0.0000,0.000,1.0000,36.0000
50%,130000.0000,0.0000,5.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.000,2.0000,...,1.0000,3.0000,1.0000,31.0000,36.0000,16.0000,36.0000,1.000,2.2500,76.5625
75%,130000.0000,0.0000,6.0000,0.0000,1.0000,0.0000,1.0000,2.0000,1.000,3.0000,...,1.0000,4.0000,1.0000,51.0000,121.0000,25.0000,81.0000,4.000,4.0000,132.2500
max,1000000.0000,1.0000,11.0000,1.0000,1.0000,1.0000,5.0000,8.0000,6.000,8.0000,...,1.0000,10.0000,1.0000,97.0000,441.0000,169.0000,441.0000,81.000,36.0000,1369.0000


In [27]:
''' 
Now that the data is cleaned, we move onto training models.
We will use simple models:
    - stochastic gradient descent for regression.
    - support vector machine (both classifier and regression (linear and polynomial)).
    - decision trees (both classifier and regression).
    
Note that we could train both classifications and regressions because the target values are 
household poverty levels (output values are 1, 2, 3, 4).

To train the models, we need to find the right parameters for each model and make sure
the models do not overfit the data. Therefore, we use cross validation grid search.
Finding the right parameters will require trial and error.
In addition, the randomness of the models may produce different parameters if the kernel is reset.

After we train the models, we will use the ensemble method to combine all the models.
But before we do that, we have to make sure that the errors that each model makes are 
uncorrelated with one another, elsewise models will make the same mistake, allowing majorities 
for the wrong class.
'''
print ('')

In [28]:
from sklearn.model_selection import GridSearchCV

# X_Cleaned_and_transformed for X
def find_best_param_from_Grid_Search (model, param_grid, scoring, X, y):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring=scoring)
    grid_search.fit(X, y )
    return grid_search.best_params_

# scoring types
f1 = 'f1_macro' # this is the scoring type of the Kaggle Competition (only for classifiers)
mse = 'neg_mean_squared_error'

In [29]:
from sklearn.linear_model import SGDRegressor
''' 
l2: Ridge Regression (reduce overfitting)
alpha: parameter to tune for l2
eta0: Learning rate
n_iter: number of times the model runs to train
'''
sgd_reg = SGDRegressor(penalty='l2') 
sgd_param_grid = [
    {'alpha': [.00020, .00025, .00030 ], 'n_iter': [100, 150, 200], 'eta0': [.001, .005, .01] }
]

sgd_best_param = find_best_param_from_Grid_Search (sgd_reg, sgd_param_grid, mse, X_Cleaned_and_transformed, y)
sgd_best_param

{'alpha': 0.0002, 'eta0': 0.005, 'n_iter': 200}

In [30]:
sgd_reg = SGDRegressor(penalty='l2', n_iter=200 , eta0= .005, alpha= 0.0002) 
sgd_reg.fit (X_Cleaned_and_transformed, y ) 

SGDRegressor(alpha=0.0002, average=False, epsilon=0.1, eta0=0.005,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=200, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [31]:
from sklearn.svm import LinearSVR # linear support vector regression
svm_reg = LinearSVR()
svm_param_grid = [
    {'C': [.5, 1, 1.5], 'epsilon': [.1, .15, .2]}
]
svm_best_param = find_best_param_from_Grid_Search (svm_reg, svm_param_grid, mse, X_Cleaned_and_transformed, y)
svm_best_param

{'C': 0.5, 'epsilon': 0.2}

In [32]:
svm_reg = LinearSVR(C=.5, epsilon=.2) 
svm_reg.fit (X_Cleaned_and_transformed, y ) 

LinearSVR(C=0.5, dual=True, epsilon=0.2, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [33]:
from sklearn.svm import LinearSVC
svc_clf = LinearSVC(loss='hinge')
svc_param_grid = [
    {'C': [.5, 1, 1.5]}
]

svc_best_param = find_best_param_from_Grid_Search (svc_clf, svc_param_grid, f1, X_Cleaned_and_transformed, y)
svc_best_param

{'C': 1}

In [34]:
svc_clf = LinearSVC(loss='hinge', C=1) 
svc_clf.fit (X_Cleaned_and_transformed, y ) 

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [35]:
from sklearn.svm import SVC
svc_poly_clf = SVC(kernel='rbf') # polynomial classfier with radial basis function
svc_poly_param_grid = [
    {'C': [.5, 1, 1.5], 'gamma': [10, 20, 30]}
]

svc_best_param = find_best_param_from_Grid_Search (svc_poly_clf, svc_poly_param_grid, f1, X_Cleaned_and_transformed, y)
svc_best_param

{'C': 1.5, 'gamma': 10}

In [36]:
svc_poly_clf = SVC(kernel='rbf', C=1.5, gamma=10)
svc_poly_clf.fit (X_Cleaned_and_transformed, y ) 

SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=10, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [37]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_param_grid = [{
    'max_depth': [20, 25, 30], 'min_samples_leaf': [35, 40, 45],
    'max_leaf_nodes': [40, 45, 50], 'max_features': [25, 30, 35]
    }]
tree_clf_best_param = find_best_param_from_Grid_Search (tree_clf, tree_param_grid, f1, X_Cleaned_and_transformed, y)
tree_clf_best_param

{'max_depth': 30,
 'max_features': 35,
 'max_leaf_nodes': 50,
 'min_samples_leaf': 35}

In [38]:
tree_clf = DecisionTreeClassifier(max_depth = 30, max_features = 35, max_leaf_nodes=50, min_samples_leaf=35)
tree_clf.fit (X_Cleaned_and_transformed, y ) 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=30,
            max_features=35, max_leaf_nodes=50, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=35,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [39]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg_param_grid = [{
    'max_depth': [35, 40, 45], 'min_samples_leaf': [25, 30, 35],
    'max_leaf_nodes': [60, 65, 70], 'max_features': [20, 25, 30]
    }]
tree_reg_best_param = find_best_param_from_Grid_Search (tree_reg, tree_reg_param_grid, mse, X_Cleaned_and_transformed, y)
tree_reg_best_param

{'max_depth': 45,
 'max_features': 30,
 'max_leaf_nodes': 60,
 'min_samples_leaf': 25}

In [40]:
tree_reg = DecisionTreeRegressor(max_depth = 35, max_features = 30, max_leaf_nodes=65, min_samples_leaf=25)
tree_reg.fit (X_Cleaned_and_transformed, y ) 

DecisionTreeRegressor(criterion='mse', max_depth=35, max_features=30,
           max_leaf_nodes=65, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=25,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [43]:
# Now to create an ensemble.

models = [sgd_reg, svm_reg, svc_clf, svc_poly_clf, tree_clf, tree_reg]
model_names = ['sgd_reg', 'svm_reg', 'svc_clf', 'svc_poly_clf', 'tree_clf', 'tree_reg']
some_data = X_test.copy()
labels = y_test.copy()

In [65]:
def hard_vote_ensemble_clf (models, model_names, some_data):
    prediction_df = pd.DataFrame({'FILLER' : [np.nan]})
    for i, model in enumerate( models):
        model_prediction = pd.DataFrame (model.predict(some_data), 
                                         columns=[model_names[i]])
        prediction_df = pd.concat([prediction_df, model_prediction], axis=1)
    prediction_df.drop ('FILLER', axis=1, inplace=True)
    # Round the regressors (and if greater than 5, set the value to 4).
    prediction_df['Target'] = prediction_df.mean(axis=1).round()
    prediction_df[prediction_df > 4] = 4
    # get only the average (result of the ensemble), and the entire dataframe of predictions
    return prediction_df['Target'], prediction_df.round()
ensemble, entire_ensemble = hard_vote_ensemble_clf (models, model_names, X_Cleaned_and_transformed)

entire_ensemble.head()

,sgd_reg,svm_reg,svc_clf,svc_poly_clf,tree_clf,tree_reg,Target
0,3.0,3.0,4,1,2,4.0,3.0
1,4.0,4.0,4,3,4,3.0,4.0
2,4.0,4.0,4,4,4,4.0,4.0
3,3.0,3.0,4,4,4,3.0,4.0
4,4.0,4.0,4,4,4,4.0,4.0


In [80]:
residual = entire_ensemble.sub(y.reset_index().drop('index', axis=1).squeeze(), axis=0)
residual.corr()
# None of the correlations are greater than .9, so we do not need to drop any of the models.

,sgd_reg,svm_reg,svc_clf,svc_poly_clf,tree_clf,tree_reg,Target
sgd_reg,1.000000,0.895616,0.723219,0.000191,0.659851,0.727562,0.814177
svm_reg,0.895616,1.000000,0.717860,0.002278,0.682190,0.736969,0.829134
svc_clf,0.723219,0.717860,1.000000,-0.007263,0.643779,0.620562,0.831784
svc_poly_clf,0.000191,0.002278,-0.007263,1.000000,-0.009417,0.000680,0.003425
tree_clf,0.659851,0.682190,0.643779,-0.009417,1.000000,0.676182,0.795532
tree_reg,0.727562,0.736969,0.620562,0.000680,0.676182,1.000000,0.759611
Target,0.814177,0.829134,0.831784,0.003425,0.795532,0.759611,1.000000


In [81]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(ensemble, y)
# each row is the actual class, each column is the predicted class
print (conf_mx)

[[  58    0    0    0]
 [ 255  405   97   36]
 [ 274  817  546  431]
 [   4   64  312 4346]]


In [89]:
''' Now we use the ensemble classifier to predict the test set. '''
def clean_predict (some_data, Dropped_Var):
    replaced_strings_df = some_data.replace ('no', 0).replace ('yes', np.nan).drop (['Id', 'idhogar'], axis=1)
    replaced_strings_df = replaced_strings_df.fillna ({'edjefe': 1, 'edjefa': 1})
    dropped = replaced_strings_df.drop (Dropped_Var, axis=1)
    some_data_prepared = trans_pipeline.transform(dropped)
    df = pd.DataFrame (some_data_prepared, columns = dropped.columns)
    return df

def hard_vote_ensemble_clf_test (models, model_names, some_data, Dropped_Var):
    df = clean_predict (some_data, Dropped_Var)
    ensemble, entire_ensemble = hard_vote_ensemble_clf (models, model_names, df)
    return ensemble

ensemble_test = hard_vote_ensemble_clf_test (models, model_names, some_data, Dropped_Var )
conf_mx_test = confusion_matrix (ensemble_test, labels)
conf_mx_test

array([[   8,    0,    0,    0],
       [  50,   56,   16,    8],
       [  72,  162,  106,  107],
       [  34,   93,  132, 1068]])

In [91]:
''' Now to predict the final set without target labels '''
final_ensemble_predict = hard_vote_ensemble_clf_test (models, model_names, final_test_set, Dropped_Var )

In [92]:
final_predictions_df = pd.DataFrame ({
    'Id': final_test_set['Id'],
    'Target': final_ensemble_predict
})

final_predictions_df = final_predictions_df.set_index ('Id')
final_predictions_df.head()
final_predictions_df.to_csv ('test_predictions.csv')